In [1]:
# uncomment and run to install 

#!pip install yfinance 
#!pip install statsmodels

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

from scipy.stats import pearsonr, spearmanr, kendalltau

from statsmodels.tsa.stattools import adfuller

import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [3]:
frequency = "1d"
start_time = "2018-01-01"
end_time = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')  # Set end_time to yesterday

1. **TLT (iShares 20+ Year Treasury Bond ETF)**: Invests in U.S. Treasury bonds with remaining maturities greater than 20 years, aiming to track the investment results of an index composed of U.S. Treasury bonds.

2. **IEF (iShares 7-10 Year Treasury Bond ETF)**: Seeks to track the investment results of an index composed of U.S. Treasury bonds with remaining maturities between 7 and 10 years.

3. **EZU (iShares MSCI Eurozone ETF)**: Provides exposure to large and mid-sized companies in Eurozone countries, tracking the MSCI EMU Index.

4. **CNYA (iShares MSCI China A ETF)**: Offers exposure to large and mid-cap Chinese equities in the A-shares market, tracking the MSCI China A Inclusion Index.

5. **EWJ (iShares MSCI Japan ETF)**: Tracks the investment results of an index composed of Japanese equities, representing large and mid-sized companies in Japan.

6. **IGLT.L (iShares Core UK Gilts UCITS ETF)**: Invests in UK government bonds (gilts), seeking to track the performance of an index composed of sterling-denominated UK government bonds.

7. **XBB.TO (iShares Core Canadian Universe Bond Index ETF)**: Offers exposure to Canadian government and corporate bonds, tracking the performance of the broad Canadian bond market.

8. **BOND.AX (PIMCO Australian Bond Index Fund)**: Provides diversified exposure to the Australian bond market, including government, semi-government, and corporate debt securities.

9. **INDA (iShares MSCI India ETF)**: Aims to track the investment results of an index composed of Indian equities, representing large and mid-sized companies in India.

10. **EWL (iShares MSCI Switzerland ETF)**: Tracks the investment results of an index composed of Swiss equities, representing the Swiss stock market.

11. **ERUS (iShares MSCI Russia ETF)**: Seeks to track the investment results of an index composed of Russian equities.

12. **EWZ (iShares MSCI Brazil ETF)**: Aims to track the investment results of an index composed of Brazilian equities, reflecting the performance of the Brazilian stock market.

13. **EWY (iShares MSCI South Korea ETF)**: Tracks the investment results of an index composed of South Korean equities, representing the South Korean stock market.

14. **ENZL (iShares MSCI New Zealand ETF)**: Seeks to track the investment results of an index composed of New Zealand equities.

15. **EWD (iShares MSCI Sweden ETF)**: Aims to track the investment results of an index composed of Swedish equities, representing the Swedish stock market.

16. **EWM (iShares MSCI Malaysia ETF)**: Seeks to track the investment results of an index composed of Malaysian equities, representing the Malaysian stock market.

In [4]:
# Define ETFs
etfs = ["TLT", "IEF", "EZU", "CNYA", "EWJ", "IGLT.L", "XBB.TO", "BOND.AX", "INDA", "EWL", "ERUS", "EWZ", "EWY", "ENZL", "EWD", "EWM"]

# Fetch ETF data
etf_data = {etf: yf.Ticker(etf).info for etf in etfs}

# Retrieve AUM or market cap
aum = {etf: data.get('totalAssets', 0) for etf, data in etf_data.items()}  

# Calculate total AUM
total_aum = sum(aum.values())

# Calculate weightings
weights = {etf: value / total_aum for etf, value in aum.items()}

print(weights)

{'TLT': 0.4062292647451769, 'IEF': 0.21725878580669125, 'EZU': 0.055852416978285045, 'CNYA': 0.0015059291660515857, 'EWJ': 0.1100509895246457, 'IGLT.L': 0.0, 'XBB.TO': 0.05079602331873081, 'BOND.AX': 0.00028976165698744034, 'INDA': 0.06216563494820313, 'EWL': 0.010017395366789108, 'ERUS': 3.5989277115003057e-06, 'EWZ': 0.04721188425186288, 'EWY': 0.03323243466653609, 'ENZL': 0.0008995802005002043, 'EWD': 0.0026062376516634176, 'EWM': 0.0018800627901649591}


In [5]:
# Existing tickers and ETF tickers (& BTC, ETH in USD)
tickers = ["^IRX", "^TNX", "^TYX", "SPY", "QQQ"]
etf_tickers = etfs  
tickers.extend(etf_tickers)
crypto_tickers = ['ETH-USD']
tickers.extend(crypto_tickers)

# Retrieve historical data
historical_data = {}
for ticker in tickers:
    historical_data[ticker] = yf.download(ticker, start=start_time, end=end_time, interval=frequency)['Close']

# Combine and weight data
combined_data = pd.DataFrame()
for ticker, data in historical_data.items():
    if ticker in weights:  # Check if ticker is in ETFs list
        combined_data[ticker] = data * weights[ticker]
    else:  # Use the data as is for other tickers
        combined_data[ticker] = data

# Calculate the Global Liquidity Index as the sum of all (weighted and non-weighted) columns, excluding crypto tickers
combined_data['Global_Liquidity_Index'] = combined_data.drop(columns=crypto_tickers, errors='ignore').sum(axis=1)

# Count NaN values in each column
nan_count_per_column = combined_data.isna().sum()
print("NaN count per column:\n", nan_count_per_column)

# Total NaN values in the DataFrame
total_nan_count = combined_data.isna().sum().sum()
print("Total NaN values in the DataFrame:", total_nan_count)

combined_data.dropna(inplace=True)

# combined_data now contains the weighted Global Liquidity Index and individual ticker data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
df = combined_data[['Global_Liquidity_Index', 'SPY', 'ETH-USD']].copy() # truncate df for dune upload

# Calculate the first difference for the specified columns
df['GLI_diff'] = df['Global_Liquidity_Index'].diff()
df['SPY_diff'] = df['SPY'].diff()
df['ETH_diff'] = df['ETH-USD'].diff()

df.dropna(inplace=True) # drop missing value from first differencing  
df

,Global_Liquidity_Index,SPY,ETH-USD,GLI_diff,SPY_diff,ETH_diff
Date,,,,,,
2018-01-03,529.356207,270.470001,962.719971,3.621430,1.700012,78.276001
2018-01-04,530.892631,271.609985,980.921997,1.536424,1.139984,18.202026
2018-01-05,534.315815,273.420013,997.719971,3.423184,1.810028,16.797974
2018-01-08,535.440183,273.920013,1148.530029,1.124368,0.500000,150.810059
2018-01-09,535.429020,274.540009,1299.739990,-0.011163,0.619995,151.209961
...,...,...,...,...,...,...
2023-12-27,981.566617,476.510010,2378.739990,6.964180,2.860016,52.215088
2023-12-28,981.310051,476.690002,2347.566162,-0.256566,0.179993,-31.173828
2023-12-29,977.724911,475.309998,2300.690674,-3.585140,-1.380005,-46.875488


In [7]:
df.to_csv('eth_global_liquidity_index.csv') 

In [8]:
for column in df.columns:
    # Perform Augmented Dickey-Fuller test:
    print(f'Testing for stationarity in series: {column}')
    result = adfuller(df[column].dropna())  # dropna() handles missing values
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    # Interpretation
    if result[1] > 0.05:
        print(f"The series {column} is likely non-stationary (fail to reject H0)\n")
        print("___________________________________________________________________")
    else:
        print(f"The series {column} is likely stationary (reject H0)\n")
        print("___________________________________________________________________")

Testing for stationarity in series: Global_Liquidity_Index
ADF Statistic: -0.870077
p-value: 0.797667
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568
The series Global_Liquidity_Index is likely non-stationary (fail to reject H0)

___________________________________________________________________
Testing for stationarity in series: SPY
ADF Statistic: -0.911177
p-value: 0.784269
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568
The series SPY is likely non-stationary (fail to reject H0)

___________________________________________________________________
Testing for stationarity in series: ETH-USD
ADF Statistic: -1.609237
p-value: 0.478952
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568
The series ETH-USD is likely non-stationary (fail to reject H0)

___________________________________________________________________
Testing for stationarity in series: GLI_diff
ADF Statistic: -11.942660
p-value: 0.000000
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568
The series GL

## Correlation

In [9]:
corr_df = df.copy()

In [10]:
def calculate_correlations(ts1, ts2):
    """
    Calculate the Pearson, Spearman, and Kendall Tau correlation coefficients along with their p-values for two time series.

    Parameters:
    ts1 : Pandas Series
        The first time series.
    ts2 : Pandas Series
        The second time series.

    Returns:
    dict
        A dictionary containing the Pearson, Spearman, and Kendall Tau correlation coefficients and their p-values.
    """
    pearson_corr, pearson_p = pearsonr(ts1, ts2)
    spearman_corr, spearman_p = spearmanr(ts1, ts2)
    kendall_corr, kendall_p = kendalltau(ts1, ts2)

    return {
        'Pearson': {'Coefficient': pearson_corr, 'P-Value': pearson_p},
        'Spearman': {'Coefficient': spearman_corr, 'P-Value': spearman_p},
        'Kendall Tau': {'Coefficient': kendall_corr, 'P-Value': kendall_p}
    }

In [11]:
gli_eth_corr = calculate_correlations(corr_df['Global_Liquidity_Index'], corr_df['ETH-USD'])
gli_eth_corr

{'Pearson': {'Coefficient': 0.8536303299738879, 'P-Value': 0.0},
 'Spearman': {'Coefficient': 0.853231414729038, 'P-Value': 0.0},
 'Kendall Tau': {'Coefficient': 0.6608060187060053,
  'P-Value': 1.167532715211163e-308}}

In [12]:
i0_gli_eth_corr = calculate_correlations(corr_df['GLI_diff'], corr_df['ETH_diff'])
i0_gli_eth_corr

{'Pearson': {'Coefficient': 0.2728260206791386,
  'P-Value': 5.588229272732466e-26},
 'Spearman': {'Coefficient': 0.27605847906020864,
  'P-Value': 1.39160228141314e-26},
 'Kendall Tau': {'Coefficient': 0.1889805256122655,
  'P-Value': 6.592866680905039e-27}}

In [13]:
gli_spy_corr = calculate_correlations(corr_df['Global_Liquidity_Index'], corr_df['SPY'])
gli_spy_corr

{'Pearson': {'Coefficient': 0.9894756608457902, 'P-Value': 0.0},
 'Spearman': {'Coefficient': 0.989543352183409, 'P-Value': 0.0},
 'Kendall Tau': {'Coefficient': 0.9228492657878836, 'P-Value': 0.0}}

In [14]:
i0_gli_spy_corr = calculate_correlations(corr_df['GLI_diff'], corr_df['SPY_diff'])
i0_gli_spy_corr

{'Pearson': {'Coefficient': 0.9762339133756012, 'P-Value': 0.0},
 'Spearman': {'Coefficient': 0.970570608216953, 'P-Value': 0.0},
 'Kendall Tau': {'Coefficient': 0.8629422323690891, 'P-Value': 0.0}}

In [15]:
def create_correlation_dataframe(dicts, names):
    """
    Create a pandas DataFrame from a list of dictionaries containing correlation coefficients, with named rows.

    Parameters:
    dicts : list of dict
        List of dictionaries with correlation data.
    names : list of str
        List of names for each time series pair.

    Returns:
    pd.DataFrame
        A DataFrame with correlation coefficients and named rows.
    """
    # Prepare data for DataFrame
    data = {name: {metric: d[metric]['Coefficient'] for metric in ['Pearson', 'Spearman', 'Kendall Tau']} for name, d in zip(names, dicts)}

    # Create DataFrame from the data
    corr_df = pd.DataFrame(data).T

    return corr_df

# Names for each time series pair
names = ["GLI-ETH", "I0 GLI-ETH", "GLI-SPY", "I0 GLI-SPY"]

# Creating the DataFrame
corr_df = create_correlation_dataframe([gli_eth_corr, i0_gli_eth_corr, gli_spy_corr, i0_gli_spy_corr], names)

# Displaying the DataFrame
corr_df.index.name = 'time_series_pairs'
corr_df.reset_index(inplace=True)
corr_df

,time_series_pairs,Pearson,Spearman,Kendall Tau
0,GLI-ETH,0.853630,0.853231,0.660806
1,I0 GLI-ETH,0.272826,0.276058,0.188981
2,GLI-SPY,0.989476,0.989543,0.922849
3,I0 GLI-SPY,0.976234,0.970571,0.862942


In [16]:
corr_df.to_csv('eth_global-liquidity_corr.csv')

## Cointegration

In [17]:
def johansen_cointegration_test(ts1, ts2, confidence_level=0.05):
    """
    Perform the Johansen cointegration test on two time series and provide an interpretation.

    Parameters:
    ts1 : Pandas Series
        The first time series.
    ts2 : Pandas Series
        The second time series.
    confidence_level : float
        The confidence level for determining cointegration (default is 0.05).

    Returns:
    None
    """
    # Combine the series into a 2D array
    ts_combined = np.column_stack([ts1, ts2])

    # Perform the Johansen cointegration test
    result = coint_johansen(ts_combined, det_order=0, k_ar_diff=1)

    # The trace statistic and the maximum eigenvalue statistic
    trace_stat = result.lr1
    max_eig_stat = result.lr2

    # Critical values at chosen confidence level (90%, 95%, 99%)
    crit_val_index = {0.10: 0, 0.05: 1, 0.01: 2}[confidence_level]
    trace_crit_vals = result.cvt[:, crit_val_index]
    max_eig_crit_vals = result.cvm[:, crit_val_index]

    # Print results and interpretation
    print("Johansen Cointegration Test Results and Interpretation:")
    print("-------------------------------------------------------")
    for idx, (trace, max_eig, trace_crit, max_eig_crit) in enumerate(zip(trace_stat, max_eig_stat, trace_crit_vals, max_eig_crit_vals)):
        print(f"Result for the Hypothesis of {idx} cointegrating relations:")
        print(f"Trace Statistic: {trace:.2f}, Critical Value: {trace_crit:.2f}")
        print(f"Max-Eigen Statistic: {max_eig:.2f}, Critical Value: {max_eig_crit:.2f}")

        if trace > trace_crit and max_eig > max_eig_crit:
            print("Both Trace and Max-Eigen statistics are above the critical values, suggesting cointegration.")
        elif trace > trace_crit or max_eig > max_eig_crit:
            print("One of the statistics is above the critical value, suggesting possible cointegration.")
        else:
            print("Neither statistic is above the critical value, suggesting no cointegration.")
        print("")

In [18]:
gli = df['Global_Liquidity_Index']
eth = df['ETH-USD']
spy = df['SPY']

In [19]:
johansen_cointegration_test(gli, eth)

Johansen Cointegration Test Results and Interpretation:
-------------------------------------------------------
Result for the Hypothesis of 0 cointegrating relations:
Trace Statistic: 12.21, Critical Value: 15.49
Max-Eigen Statistic: 11.27, Critical Value: 14.26
Neither statistic is above the critical value, suggesting no cointegration.

Result for the Hypothesis of 1 cointegrating relations:
Trace Statistic: 0.94, Critical Value: 3.84
Max-Eigen Statistic: 0.94, Critical Value: 3.84
Neither statistic is above the critical value, suggesting no cointegration.



In [20]:
johansen_cointegration_test(gli, eth)

Johansen Cointegration Test Results and Interpretation:
-------------------------------------------------------
Result for the Hypothesis of 0 cointegrating relations:
Trace Statistic: 12.21, Critical Value: 15.49
Max-Eigen Statistic: 11.27, Critical Value: 14.26
Neither statistic is above the critical value, suggesting no cointegration.

Result for the Hypothesis of 1 cointegrating relations:
Trace Statistic: 0.94, Critical Value: 3.84
Max-Eigen Statistic: 0.94, Critical Value: 3.84
Neither statistic is above the critical value, suggesting no cointegration.

